### Installation

In [ ]:
!pip install psycopg
!pip install psycopg2
!pip install python-dotenv
!pip install openpyxl
!pip install SQLAlchemy
!pip install openAI

!sudo apt-get update
!sudo apt-get install openvpn -y




### Load credentials and Variables

In [1]:
import psycopg2
from psycopg2 import sql
import json

# Load environment variables
import os
from dotenv import load_dotenv

load_dotenv()

# Get database credentials from environment variables
dbname = os.getenv("dbname")
engine = os.getenv("engine")
host = os.getenv("host")
password = os.getenv("password")
port = os.getenv("port")
username = os.getenv("username")
vpn_path = os.getenv("vpn_path")
api_key = os.getenv("api_key")

# Database connection parameters
db_params = {
    'dbname': dbname,
    'user': username,
    'password': password,
    'host': host,
    'port': port  # Add port if necessary
}


# Load the configuration file
with open('config.json', 'r') as file:
    config = json.load(file)

# Extract schema from the config
schema = config['schema']

# Combine the lines to form the full prompt with f-string
prompt_template = "\n".join(config["prompt"])

# Format the prompt with the schema variable
base_prompt = prompt_template.format(schema=schema)

# Extract results_instructions from config
results_instructions = config['results_instructions']

# Extract main_bot_instructions from config
main_bot_instructions = config['main_bot_instructions']

import subprocess

# Step 3: Start OpenVPN in the background and check the status
openvpn_process = subprocess.Popen(['sudo', 'openvpn', '--config', vpn_path],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)

### Database Project Code Main

In [2]:
from openai import OpenAI
import json
from sqlalchemy import create_engine, text

# Initialize OpenAI client
client = OpenAI(api_key=api_key)


def get_answer_from_prompt(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": base_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_tokens=512,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    # Extract the SQL query from the response
    sql_query = response.choices[0].message.content.replace('```sql\n', '').replace('\n```', '').replace('"""sql\n', '').replace('\n"""', '')

    # Create a connection string
    connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

    # Connect to the PostgreSQL database using SQLAlchemy
    engine = create_engine(connection_string)
    with engine.connect() as conn:
        # Execute the query and fetch the results
        result = conn.execute(text(sql_query))
        rows = result.fetchall()

    # Convert the result to a list of dictionaries
    data = [dict(row._asdict()) for row in rows]

    # Add the URL to each result
    for item in data:
        if 'id' in item:
            item['URL'] = f"goweasy.com/house-details/{item['id']}"

    # Convert the data to a JSON response
    json_response = json.dumps({"results": data})
    return json_response

# Function to interpret the JSON response and generate a human-readable response
def interpret_results(prompt, json_response):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": results_instructions},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": f"Here are the results from the database query: {json_response}"}
        ],
        temperature=1,
        max_tokens=512,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

# Function to call the OpenAI API with the user's prompt and generate a human-readable response
def get_chatbot_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": main_bot_instructions},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        functions=[
            {
                "name": "get_answer_from_prompt",
                "description": "The get_answer_from_prompt function uses OpenAI's GPT-4 model to generate a SQL query based on a given prompt, executes the query on a PostgreSQL database, processes the results to include a URL for each entry, and returns the results in a JSON format.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "prompt": {
                            "type": "string",
                            "description": "The user prompt"
                        }
                    },
                    "required": ["prompt"]
                }
            }
        ]
    )

    # Check if the response includes a function call
    function_call = response.choices[0].message.function_call if hasattr(response.choices[0].message, 'function_call') else None

    # Call the function with the extracted arguments if present
    if function_call and function_call.name == "get_answer_from_prompt":
        function_args = json.loads(function_call.arguments)
        json_response = get_answer_from_prompt(function_args["prompt"])
        return interpret_results(prompt, json_response)
    else:
        return response.choices[0].message.content

### Prompt testing

In [4]:
#prompt = "How are you doing?"
#prompt = "What time is it?"
#prompt = "Find a home that fits 8 people, close to disney and has a pool."
#prompt = "Does house id 215 have a pool?"
#prompt = "Show me a house without a pool, close to UCF"
#prompt = "Does house id 215 have a pool? How many people it can fit?"
#prompt = 'Hola, como estas?'
#prompt = 'Que hora es?'
#prompt = 'Quiero un hogar donde entran 8 personas, que tenga piscina y tambien que este cerca de disney.'
#prompt = 'Find a home close to disney with a pool that also fits 8 people.'
#prompt = 'preciso de uma casa que caiba 6 pessoas, perto da disney e que seja tematica'
prompt = 'Oi eu quero viajar para orlando em janeiro de 2025 do dia 5 ate o dia 15'
print(get_chatbot_response(prompt))



Parece que você está planejando uma viagem, mas meu foco é ajudar com perguntas relacionadas ao site e ao banco de dados na área de habitação. Posso ajudar você a encontrar opções de hospedagem ou informações sobre imóveis em Orlando? Se sim, por favor, me forneça mais detalhes sobre o que você está procurando!
